In [1]:
from torch.utils.data import DataLoader
from models import ADM, UDN
from methods import ADM_Deblur, UDN_Deblur, ADM_UDN_Combine
from datasets import Set14Dataset
from utils import ratio_determine
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
task = 'Set14'
sigma = 2.55 # selected from [2.55, 7.65, 12.75, 25.5]

adm_modelsigma = 49
adm_ite = 30
udn_modelsigma = 49
udn_n_layer = 8
rho_list = sigma**2/np.logspace(np.log10(udn_modelsigma), np.log10(5), udn_n_layer)**2


adm_ratio = ratio_determine(sigma)

adm_model = ADM().to(device).eval()

udn_model = UDN(rho_list, n_layer = udn_n_layer).to(device).eval()
udn_model.load_state_dict(torch.load('models/UDN{}/model_{}_050.pth'.format(udn_n_layer, sigma))['stat_dict'])

v = Set14Dataset(sigma = sigma)
V = DataLoader(dataset = v, num_workers = 1, batch_size = 1, shuffle = False)

for id, data in enumerate(V):
    adm_result = ADM_Deblur(adm_model, sigma, task_name = task, data = data, \
                        modelSigma1 = adm_modelsigma, modelSigma2 = 5, ite = adm_ite, early = -1, base = 30,\
                        u_thre = 0.05, l_thre = 0.05, ratio = adm_ratio)
    
    udn_result = UDN_Deblur(udn_model, data, ratio = adm_ratio, n_layer = udn_n_layer) 
    
    adm_udn_result = ADM_UDN_Combine(data, adm_result['Z-sub'][-1], udn_result['Z-sub'][-1])

ADM deblur elapse time: 0.5363sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.38/0.801
UDN deblur telapse time: 0.1605sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.68/0.812
ADM_UDN deblur elapse time: 0.0113sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.77/0.814
ADM deblur elapse time: 0.8028sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 31.44/0.906
UDN deblur telapse time: 0.2537sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 30.61/0.888
ADM_UDN deblur elapse time: 0.0126sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 31.32/0.905
ADM deblur elapse time: 0.5359sec
task: test-g02-k02 blurry: 21.67/0.453 restored: 27.82/0.822
UDN deblur telapse time: 0.1659sec
task: test-g02-k02 blurry: 21.67/0.453 restored: 28.07/0.831
ADM_UDN deblur elapse time: 0.0156sec
task: test-g02-k02 blurry: 21.67/0.453 restored: 28.09/0.832
ADM deblur elapse time: 0.2526sec
task: test-g03-k03 blurry: 18.72/0.299 restored: 29.19/0.783
UDN deblur telapse time: 0.0867sec
